<a href="https://colab.research.google.com/github/chaenii989/Project_2_Impacts_on_US_Housing/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning and Transforming Housing Data

## Import and Read

In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import datetime as dt
from functools import reduce

In [2]:
# Read csv

# 30-year fixed rate mortgage quarterly average (FRED)
# 1990 to 2021, float, rounding required, remove 7/1/2021 (just showing ".")
interest_rate_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/MORTGAGE30US.csv')

# Housing authorized, construction started
# 1990 to 2021, convert to integers
authorized_started_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/authorized_started_HOUST.csv')

# Average home price
# 1963 to 2021, convert to integers
average_home_price_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/average_home_price.csv')

# Homeownership rate
# 1990 to 2021, float, nothing further needed with data type/rounding
homeownership_rate_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/homeownership_rate_RSAHORUSQ156S.csv')

# Lumber price index
# 1990 to 2021, float, rounding required
lumber_price_index_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/lumber_price_19900101_20210601.csv')

# House supply
# 1990 to 2021, float, nothing further needed wiht data type/rounding
house_supply_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/monthly_supply_houses_us_MSACSR.csv')

# New housing permits
# 1990 to 2021, convert to integers
new_housing_permits_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/new_housing_permits_PERMIT.csv')

# Steel price index
# 1990 to 2021, float, rounding required
steel_price_index_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/steel_price_index.csv')

# Under construction
# 1990 to 2021, convert to integers
under_construction_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/under_counstruction_UNDCONTSA.csv')

# Units completed
# 1990 to 2021, convert to integers
units_completed_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/units_completed_COMPUTSA.csv')

## Clean Each Dataset

In [3]:
# Clean interest rate DataFrame
# Check data types
interest_rate_df.dtypes
# Change column names
interest_rate_df = interest_rate_df.rename(columns={"MORTGAGE30US":"interest_rate","DATE":"date"})
# Drop entire row for 7/1/21 which reads "." 
interest_rate_df.drop(interest_rate_df.index[(interest_rate_df["date"] == "2021-07-01")],axis=0,inplace=True)
# Convert object to float
interest_rate_df["interest_rate"] = interest_rate_df["interest_rate"].astype(float, errors = 'raise')
# Round to two decimal places
interest_rate_df["interest_rate"] = interest_rate_df["interest_rate"].round(decimals=2)
# Check for any null values in entire DF
interest_rate_df.isnull().values.any()

interest_rate_df.head()

,date,interest_rate
0,1990-01-01,10.13
1,1990-04-01,10.32
2,1990-07-01,10.10
3,1990-10-01,9.96
4,1991-01-01,9.50


In [4]:
# Clean housing authorized, started DataFrame
# Check data types
authorized_started_df.dtypes
# Change column names
authorized_started_df = authorized_started_df.rename(columns={"HOUST":"units_authorized_started","DATE":"date"})
# Check for any null values in entire DF
authorized_started_df.isnull().values.any()
authorized_started_df.drop(authorized_started_df.index[(authorized_started_df["date"] == "2021-07-01")],axis=0,inplace=True)

authorized_started_df.tail()

,date,units_authorized_started
121,2020-04-01,1085.6666666666666667
122,2020-07-01,1440.3333333333333333
123,2020-10-01,1575.3333333333333333
124,2021-01-01,1599.0000000000000000
125,2021-04-01,1586.0000000000000000


In [5]:
# Clean average home price DataFrame
# Check data types
average_home_price_df.dtypes
average_home_price_df= average_home_price_df.loc[(average_home_price_df["DATE"] >= "1990-01-01")]
# Change column names
average_home_price_df = average_home_price_df.rename(columns={"MSPUS":"avg_home_price","DATE":"date"})
# Check for any null values in entire DF
average_home_price_df.isnull().values.any()

average_home_price_df.head()

,date,avg_home_price
108,1990-01-01,123900.0
109,1990-04-01,126800.0
110,1990-07-01,117000.0
111,1990-10-01,121500.0
112,1991-01-01,120000.0


In [6]:
# Clean homeownership rate DataFrame
# Check data types
homeownership_rate_df.dtypes
# Change column names
homeownership_rate_df = homeownership_rate_df.rename(columns={"RSAHORUSQ156S":"homeownership_rate","DATE":"date"})
# Check for any null values in entire DF
homeownership_rate_df.isnull().values.any()

homeownership_rate_df.head()

,date,homeownership_rate
0,1990-01-01,64.1
1,1990-04-01,63.9
2,1990-07-01,63.9
3,1990-10-01,64.0
4,1991-01-01,64.0


In [7]:
# Clean average home price DataFrame
# Check data types
average_home_price_df.dtypes
# Change column names
average_home_price_df = average_home_price_df.rename(columns={"MSPUS":"avg_home_price","DATE":"date"})
# Convert object to integer
average_home_price_df["avg_home_price"] = average_home_price_df["avg_home_price"].astype(int, errors = 'raise')
# Check for any null values in entire DF
average_home_price_df.isnull().values.any()

average_home_price_df.head()

,date,avg_home_price
108,1990-01-01,123900
109,1990-04-01,126800
110,1990-07-01,117000
111,1990-10-01,121500
112,1991-01-01,120000


In [8]:
# Clean lumber price index DataFrame
# Check data types
lumber_price_index_df.dtypes
# Change column names
lumber_price_index_df = lumber_price_index_df.rename(columns={"lumber_price":"lumber_price_index"})
# Round to two decimal places
lumber_price_index_df["lumber_price_index"] = lumber_price_index_df["lumber_price_index"].round(decimals=2)
# Check for any null values in entire DF
lumber_price_index_df.isnull().values.any()

lumber_price_index_df.head()

,date,lumber_price_index
0,1990-01-01,55.58
1,1990-04-01,56.64
2,1990-07-01,54.72
3,1990-10-01,51.38
4,1991-01-01,51.48


In [9]:
# Clean house supply DataFrame
# Check data types
house_supply_df.dtypes
# Change column names
house_supply_df = house_supply_df.rename(columns={"MSACSR":"house_supply","DATE":"date"})
# Check for any null values in entire DF
house_supply_df.isnull().values.any()

house_supply_df.head()

,date,house_supply
0,1990-01-01,7.466667
1,1990-04-01,8.133333
2,1990-07-01,8.133333
3,1990-10-01,8.466667
4,1991-01-01,8.200000


In [10]:
# Clean new housing permits DataFrame
# Check data types
new_housing_permits_df.dtypes
# Change column names
new_housing_permits_df = new_housing_permits_df.rename(columns={"PERMIT":"new_housing_permits","DATE":"date"})
# Check for any null values in entire DF
new_housing_permits_df.isnull().values.any()

new_housing_permits_df.head()

,date,new_housing_permits
0,1990-01-01,1441.000000
1,1990-04-01,1103.666667
2,1990-07-01,1041.000000
3,1990-10-01,909.000000
4,1991-01-01,850.000000


In [11]:
# Clean steel price index DataFrame
# Check data types
steel_price_index_df.dtypes
# Change column names
steel_price_index_df = steel_price_index_df.rename(columns={"WPU101":"steel_price_index","DATE":"date"})
# Round to two decimal places
steel_price_index_df["steel_price_index"] = steel_price_index_df["steel_price_index"].round(decimals=2)
# Check for any null values in entire DF
steel_price_index_df.isnull().values.any()

steel_price_index_df.head() 

,date,steel_price_index
0,1990-01-01,116.47
1,1990-04-01,117.47
2,1990-07-01,117.87
3,1990-10-01,117.07
4,1991-01-01,116.47


In [12]:
# Clean under construction DataFrame
# Check data types
under_construction_df.dtypes
# Change column names
under_construction_df = under_construction_df.rename(columns={"UNDCONTSA":"under_construction","DATE":"date"})
# Check for any null values in entire DF
under_construction_df.isnull().values.any()
under_construction_df.drop(under_construction_df.index[(under_construction_df["date"] == "2021-07-01")],axis=0,inplace=True)


under_construction_df.head()

,date,under_construction
0,1990-01-01,891.3333333333333333
1,1990-04-01,858.6666666666666667
2,1990-07-01,812.3333333333333333
3,1990-10-01,756.3333333333333333
4,1991-01-01,701.3333333333333333


In [13]:
# Clean units completed DataFrame
# Check data types
units_completed_df.dtypes
# Change column names
units_completed_df = units_completed_df.rename(columns={"COMPUTSA":"units_completed","DATE":"date"})
# Check for any null values in entire DF
units_completed_df.isnull().values.any()
units_completed_df.drop(units_completed_df.index[(units_completed_df["date"] == "2021-07-01")],axis=0,inplace=True)


units_completed_df.head()

,date,units_completed
0,1990-01-01,1401.6666666666666667
1,1990-04-01,1315.3333333333333333
2,1990-07-01,1304.6666666666666667
3,1990-10-01,1234.3333333333333333
4,1991-01-01,1138.3333333333333333


## Merge All .csv Files

In [14]:
dfs_to_merge = [interest_rate_df, 
                authorized_started_df, 
               average_home_price_df, 
               homeownership_rate_df, 
               lumber_price_index_df, 
               house_supply_df,
               new_housing_permits_df,
               steel_price_index_df, 
               under_construction_df, 
               units_completed_df
              ]

In [15]:
housing_combined_df = reduce(lambda  left,right: pd.merge(left,right,on=["date"],
                                            how="outer"), dfs_to_merge)
housing_combined_df.head()

,date,interest_rate,units_authorized_started,avg_home_price,homeownership_rate,lumber_price_index,house_supply,new_housing_permits,steel_price_index,under_construction,units_completed
0,1990-01-01,10.13,1425.6666666666666667,123900.0,64.1,55.58,7.466667,1441.000000,116.47,891.3333333333333333,1401.6666666666666667
1,1990-04-01,10.32,1212.3333333333333333,126800.0,63.9,56.64,8.133333,1103.666667,117.47,858.6666666666666667,1315.3333333333333333
2,1990-07-01,10.10,1132.0000000000000000,117000.0,63.9,54.72,8.133333,1041.000000,117.87,812.3333333333333333,1304.6666666666666667
3,1990-10-01,9.96,1042.6666666666666667,121500.0,64.0,51.38,8.466667,909.000000,117.07,756.3333333333333333,1234.3333333333333333
4,1991-01-01,9.50,894.6666666666666667,120000.0,64.0,51.48,8.200000,850.000000,116.47,701.3333333333333333,1138.3333333333333333


In [18]:
# Check data types
housing_combined_df.dtypes
# Convert object to date
housing_combined_df["date"] = pd.to_datetime(housing_combined_df["date"]).dt.normalize()
# Convert objects to integer

housing_combined_df["units_authorized_started"] = housing_combined_df["units_authorized_started"].astype(pd.Int32Dtype())
housing_combined_df["avg_home_price"] = housing_combined_df["avg_home_price"].astype(pd.Int32Dtype())
housing_combined_df["new_housing_permits"] = housing_combined_df["new_housing_permits"].astype(pd.Int32Dtype())
housing_combined_df["under_construction"] = housing_combined_df["under_construction"].astype(pd.Int32Dtype())
housing_combined_df["units_completed"] = housing_combined_df["units_completed"].astype(pd.Int32Dtype())
housing_combined_df.dtypes
housing_combined_df.head()

TypeError: cannot safely cast non-equivalent float64 to int32

In [19]:
# Sort by date
housing_combined_df = housing_combined_df.sort_values(by="date", ascending=True)
housing_combined_df

,date,interest_rate,units_authorized_started,avg_home_price,homeownership_rate,lumber_price_index,house_supply,new_housing_permits,steel_price_index,under_construction,units_completed
0,1990-01-01,10.13,1425.666667,123900,64.1,55.58,7.466667,1441.000000,116.47,891.3333333333333333,1401.6666666666666667
1,1990-04-01,10.32,1212.333333,126800,63.9,56.64,8.133333,1103.666667,117.47,858.6666666666666667,1315.3333333333333333
2,1990-07-01,10.10,1132.000000,117000,63.9,54.72,8.133333,1041.000000,117.87,812.3333333333333333,1304.6666666666666667
3,1990-10-01,9.96,1042.666667,121500,64.0,51.38,8.466667,909.000000,117.07,756.3333333333333333,1234.3333333333333333
4,1991-01-01,9.50,894.666667,120000,64.0,51.48,8.200000,850.000000,116.47,701.3333333333333333,1138.3333333333333333
...,...,...,...,...,...,...,...,...,...,...,...
121,2020-04-01,3.24,1085.666667,322600,68.1,100.00,5.400000,1212.000000,203.90,1186.6666666666666667,1204.0000000000000000
122,2020-07-01,2.95,1440.333333,337500,67.3,145.00,3.533333,1551.000000,202.17,1209.6666666666666667,1327.3333333333333333
123,2020-10-01,2.76,1575.333333,358700,65.7,147.59,3.766667,1683.000000,216.40,1246.3333333333333333,1328.6666666666666667
124,2021-01-01,2.88,1599.000000,347500,65.6,183.76,4.100000,1788.000000,266.97,1293.3333333333333333,1390.6666666666666667


In [20]:
# Create new csv
housing_combined_df.to_csv('etl/housing_combined_data.csv', index=False)